In [2]:
import sys

sys.path.append(r'C:\repos\ra2ce_multi_network') # path to the package
sys.path.append(r'C:\repos\ra2ce') # path to the package

import geopandas as gpd
import pickle

import pyproj
from networkx import MultiDiGraph
from pathlib import Path
from snkit import Network

from ra2ce.graph.exporters.multi_graph_network_exporter import MultiGraphNetworkExporter
from ra2ce.graph.exporters.network_exporter_base import NetworkExporterBase
from ra2ce.graph.network_config_data.network_config_data import NetworkConfigData, NetworkSection
from ra2ce.graph.network_wrappers.osm_network_wrapper.osm_network_wrapper import OsmNetworkWrapper
from ra2ce.graph.origins_destinations import add_od_nodes
from ra2ce_multi_network.simplify_rail import _nx_to_network, _network_to_nx
import ra2ce.graph.networks_utils as nut

c:\Users\asgarpou\AppData\Local\miniconda3\envs\ra2ce_multi_network_env\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\asgarpou\AppData\Local\Temp\ipykernel_22456\4176253495.py:6: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-up

In [3]:
# Defining ini variables
root_folder = Path(
    r'C:\Users\asgarpou\OneDrive - Stichting Deltares\Documents\Projects\Long involvement\SITO_Netwerk to system\project_files\get_shortest_routes_road')
study_area_suffix = '_GR'
output_name = f'study_area{study_area_suffix}'
polygon_path = root_folder.joinpath(f'static/network/polygon{study_area_suffix}.geojson')
od_file = root_folder.joinpath('static/network/od_nodes.geojson')

In [ ]:
# Load complex graph

od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'static/output_graph/road_complex_graph_origin_destinations_mapped_{output_name}.p'), 'rb') as f:
    complex_graph = pickle.load(f)

In [ ]:
# Simplify complex graph => get_network procedure

# simple_graph, complex_graph, link_tables = nut.create_simplified_graph(complex_graph)

# Save the link tables linking complex and simple IDs

# osm_network_wrapper._export_linking_tables(link_tables)
# if not osm_network_wrapper.is_directed and isinstance(simple_graph, MultiDiGraph):
#     simple_graph = simple_graph.to_undirected()

# Check if all geometries between nodes are there, if not, add them as a straight line.

# simple_graph = nut.add_missing_geoms_graph(simple_graph, geom_name="geometry")
# simple_graph.graph["crs"] = pyproj.CRS("EPSG:4326")

In [23]:
# Map origin-destinations the simple graphs

# simple_graph = add_od_nodes(od=gpd.read_file(od_file), graph=simple_graph, crs=pyproj.CRS("EPSG:4326"))[1]

In [ ]:
# Save the outputs

# od_gdf.to_file(od_file, driver="GeoJSON")

# simple_graph_exporter = MultiGraphNetworkExporter(
#     basename=f'road_simple_graph_origin_destinations_mapped_{output_name}',
#     export_types=['pickle', 'shp'])
# simple_graph_exporter.export_to_shp(output_dir=root_folder.joinpath(f'static/output_graph'),
#                                     export_data=simple_graph)
# simple_graph_exporter.export_to_pickle(output_dir=root_folder.joinpath(f'static/output_graph'),
#                                        export_data=simple_graph)

In [14]:
# Load graphs and networks prepared befor

od_gdf = gpd.read_file(od_file)

with open(root_folder.joinpath(f'static/output_graph/road_simple_graph_origin_destinations_mapped_{output_name}.p'), 'rb') as f:
    simple_graph = pickle.load(f)

simple_road_network = Network(
        nodes=_nx_to_network(simple_graph).nodes,
        edges=_nx_to_network(simple_graph).edges
    )

In [4]:
# Load graphs and networks prepared befor

simple_road_network = Network(
    nodes=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_simple_graph_origin_destinations_mapped_{output_name}_nodes.gpkg')),
    edges=gpd.read_file(root_folder.joinpath(f'static/output_graph/road_simple_graph_origin_destinations_mapped_{output_name}_edges.gpkg'))
)

DriverError: C:\Users\asgarpou\OneDrive - Stichting Deltares\Documents\Projects\Long involvement\SITO_Netwerk to system\project_files\get_shortest_routes_road\road_complex_graph_origin_destinations_mapped_study_area_GR_nodes.gpkg: No such file or directory

In [13]:
# Visualise

road_map = simple_road_network.edges.explore(tiles="CartoDB positron")
road_map = od_gdf.explore(m=road_map, color='springgreen')
simple_road_network.nodes[~simple_road_network.nodes['od_id'].isna()].explore(m=road_map, color='darkgreen')